##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting Started with Gemma 2 and LlamaCpp

[Gemma](https://ai.google.dev/gemma) is a family of lightweight, state-of-the-art open-source language models from Google. Built from the same research and technology used to create the Gemini models, Gemma models are text-to-text, decoder-only large language models (LLMs), available in English, with open weights, pre-trained variants, and instruction-tuned variants.
Gemma models are well-suited for various text-generation tasks, including question-answering, summarization, and reasoning. Their relatively small size makes it possible to deploy them in environments with limited resources such as a laptop, desktop, or your cloud infrastructure, democratizing access to state-of-the-art AI models and helping foster innovation for everyone.

[llama.cpp](https://github.com/ggerganov/llama.cpp) is a C++ implementation of Meta AI's LLaMA and other large language model architectures, designed for efficient performance on local machines or within environments like Google Colab. It enables you to run large language models without needing extensive computational resources.

To make working with llama.cpp more accessible,
[llama-cpp-python](https://github.com/abetlen/llama-cpp-python) provides Python bindings for the C++ library. This allows you to enjoy the performance optimizations of `llama.cpp` while benefiting from the simplicity and flexibility of Python. With llama-cpp-python, you get a convenient API for loading models, generating text, and customizing inference parameters.

In this notebook, you will learn how to run Gemma 2 models using `llama.cpp` in a Google Colab environment. You'll install the necessary packages, set up the model, and run a sample prompt.

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Gemma/Using_Gemma_with_LlamaCpp.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

## Setup

### Select the Colab runtime
To complete this tutorial, you'll need to have a Colab runtime with sufficient resources to run the Gemma model. In this case, you can use a T4 GPU:

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.

### Gemma setup

**Before you dive into the tutorial, let's get you set up with Gemma:**

1. **Hugging Face Account:**  If you don't already have one, you can create a free Hugging Face account by clicking [here](https://huggingface.co/join).
2. **Gemma Model Access:** Head over to the [Gemma model page](https://huggingface.co/collections/google/gemma-2-release-667d6600fd5220e7b967f315) and accept the usage conditions.
3. **Colab with Gemma Power:**  For this tutorial, you'll need a Colab runtime with enough resources to handle the Gemma 2B model. Choose an appropriate runtime when starting your Colab session.
4. **Hugging Face Token:**  Generate a Hugging Face access (preferably `write` permission) token by clicking [here](https://huggingface.co/settings/tokens). You'll need this token later in the tutorial.

**Once you've completed these steps, you're ready to move on to the next section where you'll set up environment variables in your Colab environment.**


### Configure your HF token

Add your Hugging Face token to the Colab Secrets manager to securely store it.

1. Open your Google Colab notebook and click on the 🔑 Secrets tab in the left panel. <img src="https://storage.googleapis.com/generativeai-downloads/images/secrets.jpg" alt="The Secrets tab is found on the left panel." width=50%>
2. Create a new secret with the name `HF_TOKEN`.
3. Copy/paste your token key into the Value input box of `HF_TOKEN`.
4. Toggle the button on the left to allow notebook access to the secret.


In [ ]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

### Install dependencies

You'll need to install a few Python packages and dependencies to interact with HuggingFace along with `llama-cpp-python` and run the model. Find some of the releases supporting CUDA 12.2 [here](https://abetlen.github.io/llama-cpp-python/whl/cu122/llama-cpp-python/).

Run the following cell to install or upgrade it:

In [ ]:
# The huggingface_hub library allows us to download models and other files from Hugging Face.
!pip install --upgrade -q huggingface_hub

# The llama-cpp-python library allows us to leverage GPUs
!pip install llama-cpp-python==0.2.90 \
  -q -U --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122

### Logging into Hugging Face Hub

Next, you'll have to log into the Hugging Face Hub using your access token. This will allow us to download the Gemma model.

In [ ]:
from huggingface_hub import login

login(os.environ["HF_TOKEN"])

### Downloading the Gemma 2 Model
Once you're logged in, you can download the Gemma 2 model files from Hugging Face. The [Gemma 2 model](https://huggingface.co/google/gemma-2-2b-GGUF) is available in **GGUF** format, which is optimized for use with `llama.cpp` and compatible tools like Llamafile.

In [ ]:
from huggingface_hub import hf_hub_download

# Specify the repository and filename
repo_id = 'google/gemma-2-2b-GGUF'  # Repository containing the GGUF model
filename = '2b_pt_v2.gguf'  # The GGUF model file

# Download the model file to the current directory
hf_hub_download(repo_id=repo_id, filename=filename, local_dir='.')

### Running the Gemma 2 model using `llama.cpp`

It's possible to run Gemma 2 models directly using the `llama.cpp` binary without relying on Python bindings. This approach allows you to work directly at the C++ level and can be useful for integration into non-Python applications or for performance optimizations.

First, you need to download the `llama.cpp` [binary](https://github.com/ggerganov/llama.cpp/releases/tag/b3496) that supports Gemma 2.

Note: The downloaded binary does not come with GPU support.

In [ ]:
# Download the prebuilt llama.cpp binary (with Gemma 2 support)
!wget -O llama.cpp.zip https://github.com/ggerganov/llama.cpp/releases/download/b3496/llama-b3496-bin-ubuntu-x64.zip
!mkdir llama.cpp && unzip -qq llama.cpp.zip -d llama.cpp

Now, you can use the `llama.cpp` binary to run the Gemma 2 model with a sample prompt.

In [ ]:
# Define your prompt
prompt = "Q: I have 3 apples and I eat 2. How many apples do I have left?\nA:"
model_path = "2b_pt_v2.gguf"

# Run the model using llama.cpp
!./llama.cpp/build/bin/llama-cli -m "{model_path}" -p "{prompt}" -n 3 \
  --color --chat-template gemma --temp 0.7

Generating the response can take some time, but soon you'll learn how to use the `llama-cpp-python` Python library to utilize Colab's built-in GPU.

### Running the Gemma 2 model using `llama-cpp-python`

You will initialize the Llama model using the `llama-cpp-python` library by loading a pre-trained Gemma 2 model from HuggingFace. Here's what each part of the code does:

- `model_path`: Path to the model.
- `verbose`: Disables verbose logging during model loading for cleaner output.
- `n_gpu_layers`: Configures GPU acceleration. A value of `-1` means it will use as many GPU layers as possible.
- `chat_format`: Sets the expected input and output format to match the Gemma model's chat format.

This sets up the environment to use the Gemma 2 language model with `llama.cpp` by loading the model weights from HuggingFace. It prepares the model for generating text in a format compatible with Gemma's specifications.

In [ ]:
from llama_cpp import Llama

llm = Llama(
    model_path=model_path,
    verbose=False,
    n_gpu_layers=-1,
    chat_format="gemma",
)

### Interacting with the Gemma model
Finally, you prompt the Gemma 2 model and generate the response.

To do this and stream the output as it's generated, set `stream=True` when calling the `llm` function and iterate over the responses using a `for` loop; using `end=''` in the `print` function ensures the text streams without adding new lines after each chunk, and setting `flush=True` forces the print function to update the output immediately, providing smooth streaming.

In [ ]:
prompt = "What are large language models?"

# Increase max_tokens if you need longer responses
output = llm(prompt, max_tokens=256, temperature=0.7, stream=True)

# Iterate over the streaming response and print them
for response in output:
    print(response['choices'][0]['text'], end='', flush=True)

Congratulations! You've successfully set up the Gemma 2 model using `llama.cpp` in a Colab environment. You can now experiment with the model, generate text, and explore its capabilities.